# Tensorboard简介与安装

Tensorboard：TensorFlow中强大的可视化工具，支持标量，图像，文本，音频，视频和Embedding等多种数据可视化：
<img style="float: center;" src="images/140.png" width="70%">

运行机制：先从Python脚本中记录可视化的数据，然后生成eventfile文件存储到硬盘，最后从终端运行Tensorboard，打开Web页面，读取存储的eventfile在web页面上进行数据的可视化。
<img style="float: center;" src="images/141.png" width="70%">

In [1]:
# 编写python脚本文件，记录可视化数据
import numpy as np
from torch.utils.tensorboard import SummaryWriter

# 创建一个writer，记录想要的可视化数据
writer = SummaryWriter(comment='test_tensorboard')

for x in range(100):

    writer.add_scalar('y=2x', x * 2, x)
    writer.add_scalar('y=pow(2, x)',  2 ** x, x)
    
    writer.add_scalars('data/scalar_group', {"xsinx": x * np.sin(x),
                                             "xcosx": x * np.cos(x),
                                             "arctanx": np.arctan(x)}, x)
writer.close()

运行结束后，可以发现当前文件夹下有一个runs：
<img style="float: center;" src="images/142.png" width="70%">

回到终端，输入tensorboard读取这个event files：
<img style="float: center;" src="images/143.png" width="70%">

点击链接进入tensorboard的界面：
<img style="float: center;" src="images/144.png" width="70%">

# Tensorboard基本使用

准确率和损失的可视化，参数数据的分布及参数梯度的可视化

## SummaryWriter

提供创建event file的高级接口

```python
class SummaryWriter(object):
    def __init__(self, log_dir=None, comment='', purge_step=None, max_queue=10, flush_secs=120, filename_suffix='')
```
- log_dir：event file输出文件夹，如果不设置，就创建一个runs
- comment：不指定log_dir时，文件夹后缀
- filename_suffix：event file文件夹后缀

先不指定log_dir：
<img style="float: center;" src="images/145.png" width="70%">

指定log_dir，发现comment就不起作用：
<img style="float: center;" src="images/146.png" width="70%">

### add_scalar()/add_scalars()

记录标量

add_scalar(tag, scalar_value, global_step=None, walltime=None)

add_scalars(main_tag, tag_scalar_dict, global_step=None, walltime=None)

- tag：图像的标签名，图的唯一标识，就是图的标题
- scalar_value：要记录的标量，可以理解为y轴
- global_step：x轴

注意：scalar_value的局限性就是它只能画一条线，但是往往模型训练的时候想监控训练集和验证集的曲线对比情况，那时候这个不能用了。

add_scalars()：
- main_tag：该图的标签
- tag_scalar_dict：key是变量tag（类似于每条曲线的标签），value是变量的值（等同于上面的scalar_value，但可以多个线）

In [2]:
writer.add_scalar('y=2x', x * 2, x)
writer.add_scalar('y=pow(2, x)',  2 ** x, x)
    
writer.add_scalars('data/scalar_group', {"xsinx": x * np.sin(x),
                                         "xcosx": x * np.cos(x),
                                         "arctanx": np.arctan(x)}, x)

结果：
<img style="float: center;" src="images/147.png" width="70%">

### add_histogram()

统计直方图与多分位数直线图，对参数的分布以及梯度的分布非常有用

add_histogram(tag, values, global_step=None, bins='tensorflow', walltime=None)

- tag：图像的标签名字，图的唯一标识
- values：统计的参数
- global_step：y轴
- bins：取直方图的bins

<img style="float: center;" src="images/148.png" width="70%">

Tensorboard中的结果：
<img style="float: center;" src="images/149.png" width="70%">

Tensorboard中的多分位折线图（可以观察每个数据方差的变化）：
<img style="float: center;" src="images/150.png" width="70%">

### 模型训练监控

采用上面两个方法进行模型训练过程中loss和acc的监控和参数的分布以及参数对应的梯度的一个分布，在具体模型训练中如何使用？

首先在训练中构建一个SummaryWriter：
<img style="float: center;" src="images/151.png" width="70%">

如何绘制训练过程中的损失，正确率变化曲线以及参数的分布及它们权重的分布：
<img style="float: center;" src="images/152.png" width="70%">

绘制验证集上的损失和正确率的曲线变化图像：
<img style="float: center;" src="images/153.png" width="70%">

最后的结果：
<img style="float: center;" src="images/154.png" width="70%">

上述是模型训练过程中学习曲线ed可视化，下面看看参数的分布直方图：
<img style="float: center;" src="images/155.png" width="70%">

以上就是如何用SummaryWriter去构建event file的一个路径，设置路径，然后add_scalar和add_histogram方法。

采用这两个方法可以监控模型训练过程中训练集和验证集loss曲线及准确率曲线对比，还有模型参数的数据分布以及每个epoch梯度更新的一个分布

## Tensorboard图像可视化方法

### add_image()

记录图像

add_image(tag, img_tensor, global_step=None, walltime=None, dataformats='CHW')

- tag：图像标签名，图的唯一标识
- img_tensor：图像数据，注意尺度（如果图片像素值为0-1，则默认会在这个基础上\*255来可视化，因为图片都是0-255，如果像素值有大于1的，则机器就会认为是0-255范围，不做任何改动）
- global_step：x轴
- dataformats：数据形式（CHW，HWC，HW灰度图）

In [3]:
import torch
import time

writer = SummaryWriter(comment='test_your_comment', filename_suffix="_test_your_filename_suffix")

# img 1，random
fake_img = torch.randn(3, 512, 512) # CHW
writer.add_image("fake_img", fake_img, 1)
time.sleep(1)

# img 2，ones
# 这个全1， 没有大于1的，所以机器会先乘以255然后显示
fake_img = torch.ones(3, 512, 512)   
writer.add_image("fake_img", fake_img, 2)
time.sleep(1)

# img 3，1.1
# 这个像素都大于1， 所以默认不处理
fake_img = torch.ones(3, 512, 512) * 1.1
writer.add_image("fake_img", fake_img, 3)
time.sleep(1)

# img 4，HW
# 灰度图像
fake_img = torch.rand(512, 512)
writer.add_image("fake_img", fake_img, 4, dataformats="HW")
time.sleep(1)

# img 5，HWC
# 演示一下dataformats
fake_img = torch.rand(512, 512, 3)
writer.add_image("fake_img", fake_img, 5, dataformats="HWC")
time.sleep(1)

writer.close()

看一下效果：
<img style="float: center;" src="images/156.png" width="70%">

上面的图片中可视化了5张图片，但是显示的时候，需要拖动去一次次地显示每一张图片，这样就无法同时对比，如果想从一个界面里同时显示多张图片？则需要用到下面地方法

### torchvision.utils.make_grid

制作网格图像

make_grid(tensor, nrow=8, padding=2, normalize=False, range=None, scale_each=False, pad_value=0)

- tensor：图像数据，B\*CHW形式，B表示图片个数
- nrow：行数（列数自动计算），根据上面指定地B来计算列数
- padding：图像间距（像素单位）
- normalize：是否将像素值标准化，视觉像素0-255，因此如果像素值是0-1地数，则将这个设置为True，就会把像素值映射到0-255之间，设置为False，则不变。（这里的标准化是针对视觉像素正常范围来讲）
- range：标准化范围，舍弃一些过大或者过小的像素，例如一张图片的像素值范围[-1000,2000]，如果指定这里的标准化范围是[-600, 500]，则会先把图片像素值规范到这个指定区间，小于-600的用-600表示，大于500的用500表示，然后在进行标准化到0-255
- scale_each：是否单张图维度标准化（有的图像可能尺度不一样，如果设置False，是从整个大张量上进行标准化）
- pad_value：padding的像素值（网格线的颜色，通常默认0）

In [7]:
import os

writer = SummaryWriter(comment='test_your_comment', filename_suffix="_test_your_filename_suffix")

split_dir = os.path.join("..", "05数据读取机制", "data", "rmb_split")
train_dir = os.path.join(split_dir, "train")
# train_dir = "path to your training data"

transform_compose = transforms.Compose([transforms.Resize((32, 64)), transforms.ToTensor()])
train_data = RMBDataset(data_dir=train_dir, transform=transform_compose)
train_loader = DataLoader(dataset=train_data, batch_size=16, shuffle=True)
data_batch, label_batch = next(iter(train_loader))

img_grid = vutils.make_grid(data_batch, nrow=4, normalize=True, scale_each=True)
# img_grid = vutils.make_grid(data_batch, nrow=4, normalize=False, scale_each=False)
writer.add_image("input img", img_grid, 0)

writer.close()

NameError: name 'transforms' is not defined

可以看到效果：
<img style="float: center;" src="images/157.png" width="70%">

add_image结合make_grid的使用方法比较实用，可以对数据进行一个基本的审查，快速的检查训练数据样本之间是否有交叉，这些样本的标签是否是正确的。（这样审查数据集就比较快了）

### add_graph()

可视化模型计算图

add_graph(model, input_to_model=None, verbose=False)

- model：模型（需要是nn.Module）
- input_to_model：输出给模型的数据
- verbose：是否打印计算图结构信息

计算图显示效果：
<img style="float: center;" src="images/158.png" width="70%">

In [8]:
writer = SummaryWriter(comment='test_your_comment', filename_suffix="_test_your_filename_suffix")

# 模型
fake_img = torch.randn(1, 3, 32, 32)

lenet = LeNet(classes=2)

writer.add_graph(lenet, fake_img)  # 这是可视化LeNet的计算图

writer.close()

NameError: name 'LeNet' is not defined

### torchsummary

查看模型信息，便于调试，打印模型输入输出的shape以及参数总量

summary(model, input_size, batch_size=-1, device='cuda')

- model：pytorch模型
- input_size：模型输入size
- batch_size：batch大小
- device：cuda或cpu

<img style="float: center;" src="images/159.png" width="70%">

# hook函数与CAM可视化

## hook函数

Hook函数机制：不改变模型的主体，实现额外功能，像一个挂件和挂钩。

为何需要这个东西？与PyTorch计算图机制有关，动态图的运算过程中，运算结束后，一些中间变量会被释放掉（比如特征图，非叶子节点的梯度）。但是往往需要提取这些中间变量，这时候可以用hook函数在前向/反向传播的时候，挂上一个额外的函数，通过这个额外的函数去获取这些可能被释放掉而后面又想用的这些中间变量，甚至可以通过hook函数去改变中间变量的梯度。

PyTorch提供四种hook函数：
- torch.Tensor.register_hook(hook)：针对tensor
- torch.nn.Module.register_forward_hook：后面这三个针对Module
- torch.nn.Module.register_forward_pre_hook
- torch.nn.Module.register_backward_hook

## hook函数与特征图提取

### torch.Tensor.register_hook

张量的hook函数，注册一个**反向传播**的hook函数（只有在反向传播的时候某些中间叶子节点的梯度才会被释放掉，才需要使用hook函数去保留一些中间变量的信息）

可以理解成一个钩子，用这个钩子挂一些函数到计算图上，然后去完成一些额外的功能。

可以发现允许挂的函数只有一个输入参数，不返回或者返回张量
- hook(grad)->Tensor or None

注册（挂上）：
<img style="float: center;" src="images/160.png" width="70%">

这个图在反向传播结束后，非叶子节点的梯度会被释放掉，即这里的a，b梯度，则如何保留住呢？

之前有一个retain_grad()方法，可以保留中间节点的梯度，其实这里hook也可以保留住梯度：
<img style="float: center;" src="images/161.png" width="70%">

hook函数不仅仅可以保留梯度，还可以做其他的功能。例如在反向传播中改变叶子节点w的梯度：
<img style="float: center;" src="images/162.png" width="70%">

可以看到，通过钩子的方式在计算图上挂函数然后去完成一些功能还是很方便的

### Module.register_forward_pre_hook

注册module的前向传播**前**的hook函数，允许挂的函数结构：
- hook(module, input)->None

因为它是挂在前向传播前的函数，所以这里接收参数就没有output

### Module.register_forward_hook

注册module的前向传播hook函数，他的函数定义方法如下：
- hook(module, input, output)->None
  - 这个钩子允许挂的函数有3个输入
  - module：当前网络层
  - input：当前网络层的输入数据
  - output：当前网络层的输出数据

通常使用这个函数在前向传播中获取卷积输出的一个特征图

### Module.register_backward_hook

注册module反向传播的hook函数
- hook(module, grad_input, grad_output)->Tensor or None

挂在反向传播后，因此当前的输入有三个参数，后两个是grad_input和grad_output（当前网络层输入梯度数据和输出梯度数据）

### 钩子函数调用机制

需求：有一张图片，通过两个卷积核提取特征，之后通过池化得到最后值。

如果是单传的前向传播，则传完后通过卷积之后的特征图就会被释放掉，那如何进行保留这些特征图并且后期可以用Tensorboard进行可视化呢？
<img style="float: center;" src="images/163.png" width="70%">

这时候可以采用前向传播hook函数来获取中间的这个特征图

In [9]:
import torch.nn as nn

# 定义我们的网络， 这里只有卷积核池化两个操作
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 2, 3)  # 1张图片， 2个卷积核， 3*3的
        self.pool1 = nn.MaxPool2d(2, 2)

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        return x

# 下面定义前向传播的hook函数
def forward_hook(module, data_input, data_output):
    fmap_block.append(data_output)
    input_block.append(data_input)

# 网络初始化
net = Net()
# 按照上面的图进行卷积层的网络初始化
net.conv1.weight[0].detach().fill_(1)
net.conv1.weight[1].detach().fill_(2)
net.conv1.bias.data.detach().zero_()

# 弄一个钩子挂上函数
fmap_block = list()   # 保存特征图
input_block = list()
# 这句话就把函数用钩子挂在了conv1上面，进行conv1输出的获取
net.conv1.register_forward_hook(forward_hook)

# 下面初始化一个输入
fake_img = torch.ones((1, 1, 4, 4))   # 根据上面图片初始化
output = net(fake_img)   # 前向传播

# 先不用反向传播，我们输出特征图看看
print("output shape: {}\noutput value: {}\n".format(output.shape, output))
print("feature maps shape: {}\noutput value: {}\n".format(fmap_block[0].shape, fmap_block[0]))
print("input shape: {}\ninput value: {}".format(input_block[0][0].shape, input_block[0]))	

output shape: torch.Size([1, 2, 1, 1])
output value: tensor([[[[ 9.]],

         [[18.]]]], grad_fn=<MaxPool2DWithIndicesBackward0>)

feature maps shape: torch.Size([1, 2, 2, 2])
output value: tensor([[[[ 9.,  9.],
          [ 9.,  9.]],

         [[18., 18.],
          [18., 18.]]]], grad_fn=<ThnnConv2DBackward0>)

input shape: torch.Size([1, 1, 4, 4])
input value: (tensor([[[[1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.]]]]),)


可以看结果：
<img style="float: center;" src="images/164.png" width="70%">

工作原理：

在output=net(fake_img)前打上断点，然后debug，步入，进入Module的\_\_call\_\_函数，这里调用前向传播函数：
<img style="float: center;" src="images/165.png" width="70%">

这里再一次步入，跳到自己写的前向传播函数里，这里第一个子模块就是卷积模块（就是放钩子的地方），再一次步入：
<img style="float: center;" src="images/166.png" width="70%">

又回到Module的\_\_call\_\_函数，因为子模块也是继承的Module（因为这里不仅是完成前向传播，第一个子模块是放了一个钩子的，所以\_\_call\_\_函数不止完成forward）：
<img style="float: center;" src="images/167.png" width="70%">

前向传播后，获得了中间特征图，但这一次有一个钩子放在这里，那么获取了中间特征图之后，不会返回，而是去执行钩子函数：
<img style="float: center;" src="images/168.png" width="70%">

在hook_result = hook(self, input, result)这一行再次步入，发现跳到我们自定义的hook函数中：
<img style="float: center;" src="images/169.png" width="70%">

这样就完成了中间图的存储。

上面的hook函数运行机制都是在\_\_call\_\_函数中完成（这也是Python代码高级的一个地方），它实现了一些hook机制，提供了一些额外的实现别的功能的一些接口。

简而言之：
- 首先在定义网络的时候，调用父类Module的\_\_init\_\_函数对模块进行初始化，当然这里的模块不仅指的最后的大网络，每个小的子模块也是如此，这个初始化的过程中是完成了8个参数字典的初始化。
- 在模型调用的时候，其实是在执行Module的\_\_call\_\_函数，这个函数其实是完成4部分的工作：
  - 前向传播之前的hook函数
  - 前向传播的hook函数
  - forward_hooks函数
  - 反向传播的hooks函数
  
以上就是PyTorch中hook函数的一个运行机制。

In [10]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 2, 3)
        self.pool1 = nn.MaxPool2d(2, 2)

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        return x

def forward_hook(module, data_input, data_output):
    fmap_block.append(data_output)
    input_block.append(data_input)

def forward_pre_hook(module, data_input):
    print("forward_pre_hook input:{}".format(data_input))

def backward_hook(module, grad_input, grad_output):
    print("backward hook input:{}".format(grad_input))
    print("backward hook output:{}".format(grad_output))

# 初始化网络
net = Net()
net.conv1.weight[0].detach().fill_(1)
net.conv1.weight[1].detach().fill_(2)
net.conv1.bias.data.detach().zero_()

# 注册hook
fmap_block = list()
input_block = list()
net.conv1.register_forward_hook(forward_hook)
net.conv1.register_forward_pre_hook(forward_pre_hook)
net.conv1.register_backward_hook(backward_hook)

# inference
fake_img = torch.ones((1, 1, 4, 4))   # batch size * channel * H * W
output = net(fake_img)

loss_fnc = nn.L1Loss()
target = torch.randn_like(output)
loss = loss_fnc(target, output)
loss.backward()

# 观察
# print("output shape: {}\noutput value: {}\n".format(output.shape, output))
# print("feature maps shape: {}\noutput value: {}\n".format(fmap_block[0].shape, fmap_block[0]))
# print("input shape: {}\ninput value: {}".format(input_block[0][0].shape, input_block[0]))


forward_pre_hook input:(tensor([[[[1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.]]]]),)
backward hook input:(None, tensor([[[[0.5000, 0.5000, 0.5000],
          [0.5000, 0.5000, 0.5000],
          [0.5000, 0.5000, 0.5000]]],


        [[[0.5000, 0.5000, 0.5000],
          [0.5000, 0.5000, 0.5000],
          [0.5000, 0.5000, 0.5000]]]]), tensor([0.5000, 0.5000]))
backward hook output:(tensor([[[[0.5000, 0.0000],
          [0.0000, 0.0000]],

         [[0.5000, 0.0000],
          [0.0000, 0.0000]]]]),)


C:\Users\Huris\anaconda3\envs\sEMG\lib\site-packages\torch\nn\modules\module.py:1025: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


这里加了另外两个hook函数，然后把最后的输出注释，观察三个hook函数的运行顺序：
<img style="float: center;" src="images/170.png" width="70%">

### 总结

hook机制：计算图上挂一些钩子，钩子上挂一些函数，在不改变模型或者计算图的主体下，实现一些额外的功能，比如保存一些中间变量。

主要有四个hook函数：
- 第一个针对Tensor：挂在反向传播之后，用于保留中间节点的梯度或者改变一些梯度等
- 另外三个针对Module，根据挂的位置不同分为三个：
  - 挂在前向传播前的：这个接收的参数没有输出，一般用来查看输入数据的信息
  - 挂在前向传播后的：这个接收的参数就是输入和输出，一般用来存储中间特征图的信息
  - 挂在反向传播后的：查看梯度信息

hook机制的运行原理：主要在Module的\_\_call\_\_函数中，这里完成四块功能，先看有没有前向传播的钩子，然后前向传播，然后前向传播后的钩子，然后反向传播钩子。

## CAM可视化

CAM（class activation map）：类激活图，分析卷积神经网络，当卷积神经网络得到输出后，可以分析网络是关注图像的哪些部分而得到的这个结果。

可以分析出网络是否学习到了图片中物体本身的特征信息：
<img style="float: center;" src="images/171.png" width="70%">

网络最后输出是澳大利亚犬种，则网络从图像中看到什么东西才确定是这一个类呢？

这里可以通过CAM算法进行可视化，结果就是下面的图像，红色的就是网络重点关注的，在这个结果中可以发现，这个网络重点关注了狗的头部，然后判断是这样一个犬种。

CAM的基本思想：它会对网络的最后一个特征图进行加权求和，就可以得到一个注意力机制，就是卷积神经网络更关注于什么地方：
<img style="float: center;" src="images/172.png" width="70%">

可以发现实验中网络在预测是飞机的时候，其实关注的不是飞机本身，而是飞机周围的天空，发现一片蓝色，所以网络就预测是飞机。

预测汽车的时候，如果把汽车缩小，周围出现了大片蓝色，就发现网络把车也预测成了飞机。（最后一张图，竟然还预测出一个船，这个可能是因为底部的左边是蓝色，右边不是蓝色，所以网络认为这个是船）

这说明网络根本没有在学习物体本身，而是光关注物体周围的环境。

通过Grad-CAM可视化可以分析卷积神经网络学习到的特征是否真的是好的，是否真的在学习物体本身。

# 思维导图

<img style="float: center;" src="images/173.png" width="70%">